In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pandas_ta as ta
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
from tqdm import tqdm
from datasetsplit import *
from architectures import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Setup complete. Using torch {torch.__version__}({device})")
torch.manual_seed(123)
lookback = 5*24
PATH = os.path.join(os.getcwd(),"models","{}daysLoopback".format(lookback),"btc1y")

In [ ]:
btc = yf.Ticker("BTC-USD")
# historico_btc = btc.history()
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
historico_btc = yf.download(tickers = "BTC-USD", start="2023-07-01",end="2023-11-19",interval="1h")
historico_eth = yf.download(tickers = "ETH-USD", start="2023-07-01",end="2023-11-19",interval="1h")
historico_btc.drop(historico_btc.tail(1).index,inplace=True)
historico_btc.drop(columns=["Volume","Adj Close"],inplace=True)
print(historico_btc.min())
print(max(historico_btc.High))
print(min(historico_btc.Low))
historico_btc = (historico_btc-min(historico_btc.Low))*10/(max(historico_btc.High)-min(historico_btc.Low))
historico_eth = (historico_eth-min(historico_eth.Low))*10/(max(historico_eth.High)-min(historico_eth.Low))
historico_btc["Eth_close"] = historico_eth.Close
historico_btc["RSI"] = ta.rsi(historico_btc.Close, length=14)
historico_btc["MACD"] = ta.ema(historico_btc.Close, length=12)-ta.ema(historico_btc.Close, length=26)
historico_btc["Signal"] = ta.ema(historico_btc.MACD, length=9)
historico_btc["CloseVar"] = historico_btc["Close"].diff()
historico_btc['CloseVar'] = historico_btc['CloseVar'].clip(upper=0.25,lower=-0.25)
historico_btc.tail(10)

# Predicting the close Price

In [ ]:
historico_btc["Target"] = historico_btc.Close[1:]
cleandf = historico_btc.drop(columns=["Volume","Adj Close"])
cleandf.Target = cleandf.Target.shift(-1)
cleandf.dropna(inplace=True)
cleandf.head(10)

In [ ]:
full_np = cleandf.values.astype('float32')
# close_var = close_var.reshape((-1,1))
data_shape = full_np.shape
print(full_np.shape)
plt.plot(full_np)
plt.show()

In [ ]:
scaler_list = []
scaled_data = np.ones_like(full_np)
for column in range(data_shape[1]):
    sc = MinMaxScaler(feature_range=(0,1))
    full_min = full_np[:,column].min()
    full_max = full_np[:,column].max()
    scaler_list.append(MinMaxScaler(feature_range=(full_min, full_max)))
    # if column == 4:
    #     scaled_data[:,column] = full_np[:,column]
    # else:
    scaled_data[:,column] = sc.fit_transform(full_np[:,column].reshape(-1,1)).reshape(-1,)
    # training_set_scaled = close_var
    print("Column: ", column)
    print("Min: ", full_min)
    print("Max: ", full_max)
    plt.plot(scaled_data[:,column])
    plt.show()

In [ ]:
# model = LSTMOnlyLast(20,1,0).to(device)
model = LSTMLastOutput(data_shape[1]-1,15,1,0).to(device)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

In [ ]:
# train-val split for time series
train_size = int(len(scaled_data[:,0]) * 0.67)
val_size = len(scaled_data[:,0])  - train_size
train, val = scaled_data[:train_size,:], scaled_data[train_size:]
if isinstance(model,LSTMLastOutput):
    X_train, y_train = create_dataset_one_output(train, lookback=lookback)
    X_val, y_val = create_dataset_one_output(val, lookback=lookback)

print("Input train size: ", X_train.shape,"; Output train size: ", y_train.shape)
print("Input val size: ", X_val.shape,"; Output val size: ", y_val.shape)



In [ ]:
optimizer = optim.Adam(model.parameters(),lr=0.0001)
loss_fn = nn.MSELoss()
train_loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=False, batch_size=256)
n_epochs = 20000
best_loss = 100
output_length = scaled_data.shape[0]
y_full_preds = np.ones((output_length-lookback,1))*np.nan
for epoch in tqdm(range(n_epochs)):
    model.train()
    # y_pred_train = torch.zeros_like(y_train)
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Validation
    if epoch % 100 == 0:
        model.eval()
        with torch.no_grad():
            X_train_gpu = X_train.to(device)
            y_pred_train = model(X_train_gpu)
            train_rmse = np.sqrt(loss_fn(y_pred_train.cpu(), y_train))

            X_val_gpu = X_val.to(device)
            y_pred_val = model(X_val_gpu)
            val_rmse = np.sqrt(loss_fn(y_pred_val.cpu(), y_val))

            # shift train predictions for plotting
            train_plot = np.ones_like(scaled_data[:,-1]) * np.nan
            train_plot[lookback:train_size] = y_pred_train.cpu()[:, -1, :].squeeze(1)
            # train_plot[lookback:train_size] = sc_train.fit_transform(y_pred_train.cpu()[:, -1, :])

            # shift val predictions for plotting
            val_plot = np.ones_like(scaled_data[:,-1]) * np.nan
            val_plot[train_size+lookback:len(scaled_data[:,-1])] = y_pred_val.cpu()[:, -1, :].squeeze(1)
            # val_plot[train_size+lookback:len(scaled_data)] = sc_val.fit_transform(y_pred_val.cpu()[:, -1, :])
            
            # plot
            fig, ax = plt.subplots()
            ax.plot(scaled_data[:,-1], c='b')
            ax.plot(train_plot, c='r')
            ax.plot(val_plot, c='g')
            start, end = ax.get_xlim()
            ax.xaxis.set_ticks(np.arange(0, end, 30*24))
            ax.grid()
            if val_rmse < best_loss:
                best_loss = val_rmse
                try:
                    torch.save(model.state_dict(), os.path.join(PATH,"best_model.pt"))
                    plt.savefig(os.path.join(PATH,"result.png"))
                except:
                    os.makedirs(PATH)
                    torch.save(model.state_dict(), os.path.join(PATH,"best_model.pt"))
                    plt.savefig(os.path.join(PATH,"result.png"))

                print("New Best model saved")
            # plt.show()
            plt.close()

        print("Epoch %d: train RMSE %.6f, val RMSE %.6f, best val %.6f" % (epoch, train_rmse, val_rmse, best_loss))

In [ ]:
fig, ax = plt.subplots()
ax.plot(scaled_data[:,-1], c='b')
ax.plot(train_plot, c='r')
ax.plot(val_plot, c='g')
# ax.plot(full_test_plot,c='m')
start, end = ax.get_xlim()
# ax.xaxis.set_ticks(np.arange(0, end, 1))
# ax.yaxis.set_ticks(np.arange(0, 1, 0.05))
ax.grid()
plt.xlim(5700,5900)
plt.ylim(0.6,0.8)
plt.show()

## Predicción de valores futuros

In [ ]:
btc = yf.Ticker("BTC-USD")
historico_btc = btc.history(period="5y")
close_price = historico_btc.iloc[:,1:2].values.astype('float32')
close_price = close_price.reshape((-1,1))
sc = MinMaxScaler(feature_range=(0,1))
test_set_scaled = sc.fit_transform(close_price)
model = LSTMOnlyLast(20,1,0).to(device)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))



In [ ]:
train_size = int(len(test_set_scaled) * 0.67)
val_size = len(test_set_scaled) - train_size
train, val = test_set_scaled[:train_size], test_set_scaled[train_size:]
test = test_set_scaled[-lookback-1:]

lookback = 30

if isinstance(model,LSTMOnlyLast):
    X_val, y_val = create_dataset_one_output(val, lookback=lookback)
    X_full, y_full = create_single_sample(train, lookback=lookback)
    X_test = create_testset(test, lookback=lookback)

elif isinstance(model, LSTMWhole):
    X_val, y_val = create_dataset_whole_output(val, lookback=lookback)
    X_full, y_full = create_single_sample(train, lookback=lookback)
    X_test = create_testset(test, lookback=lookback)


val_min = close_price[train_size:,:].min()
val_max = close_price[train_size:,:].max()
sc_val = MinMaxScaler(feature_range=(val_min, val_max))
print(X_test.shape)
test_min = close_price[-lookback:,:].min()
test_max = close_price[-lookback:,:].max()
sc_test = MinMaxScaler(feature_range=(test_min, test_max))

In [ ]:
epoch = 300
model_path = PATH = os.path.join(os.getcwd(),"models","{}daysLoopback".format(lookback),"btc5y", "best_model.pt")
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()
n_preds = 31
y_preds = np.ones((n_preds,1))*np.nan
y_val_preds = np.ones((val_size-lookback,1))*np.nan
y_full_preds = np.ones((test_set_scaled.shape[0]-lookback,1))*np.nan

with torch.no_grad():
    X_val_gpu = X_val.to(device)
    y_pred_val = model(X_val_gpu)
    val_plot = np.ones_like(test_set_scaled) * np.nan
    val_plot[train_size+lookback:len(test_set_scaled)] = y_pred_val.cpu()[:, -1, :]
    for pred in range(test_set_scaled.shape[0]-lookback):
        X_full_gpu = X_full.to(device)
        y_pred_full = model(X_full_gpu)
        y_full_preds[pred,:] = y_pred_full.cpu()[:, -1, :]
        new_input = np.ones_like(X_full.numpy())
        new_input[:,:-1,:] = X_full.numpy()[:,1:,:]
        new_input[:,-1,:] = y_pred_full.cpu()[:, -1, :]
        X_full = torch.tensor(new_input)
        full_test_plot = np.ones((test_set_scaled.shape[0],1)) * np.nan
        full_test_plot[lookback:] = y_full_preds.reshape(-1,1)
    for n_pred in range(n_preds):
        X_test_gpu = X_test.to(device)
        y_pred_test = model(X_test_gpu)
        y_preds[n_pred,:] = y_pred_test.cpu()[:, -1, :]
        new_input = np.ones_like(X_test.numpy())
        new_input[:,:-1,:] = X_test.numpy()[:,1:,:]
        new_input[:,-1,:] = y_pred_test.cpu()[:, -1, :]
        X_test = torch.tensor(new_input)
        test_plot = np.ones((test_set_scaled.shape[0]+n_preds,1)) * np.nan
        test_plot[len(test_set_scaled):len(test_set_scaled)+n_preds] = y_preds.reshape(-1,1)
        # plot
plt.plot(test_set_scaled, c='b')
plt.plot(val_plot,c='g')
plt.plot(test_plot, c='g')
plt.plot(full_test_plot,c='m')
plt.show()

In [ ]:
plt.plot(test_set_scaled, c='b')
plt.plot(val_plot,c='g')
plt.plot(test_plot, c='g')
plt.xlim((500,600))
plt.show()

# Predicting the variation of the close price

In [ ]:
historico_btc["Target"] = historico_btc["CloseVar"]
historico_btc.Target=historico_btc.Target.shift(-1)
historico_btc.tail(5)

In [ ]:
cleandf=historico_btc
cleandf.dropna(inplace=True)
cleandf.tail(10)

In [ ]:
full_np = cleandf.values.astype('float32')
# close_var = close_var.reshape((-1,1))
data_shape = full_np.shape
print(full_np.shape)
plt.plot(full_np)
plt.show()

In [ ]:
data_shape = full_np.shape
print(full_np.shape)

In [ ]:
# scaler_list = []
# scaled_data = np.ones_like(full_np)
# for column in range(data_shape[1]):
#     sc = MinMaxScaler(feature_range=(0,100))
#     full_min = full_np[:,column].min()
#     full_max = full_np[:,column].max()
#     scaler_list.append(MinMaxScaler(feature_range=(full_min, full_max)))
#     if column in [4,5,6]:
#         scaled_data[:,column] = full_np[:,column]
#     else:
#         scaled_data[:,column] = sc.fit_transform(full_np[:,column].reshape(-1,1)).reshape(-1,)
#     # training_set_scaled = close_var
#     print("Column: ", column)
#     print("Min: ", full_min)
#     print("Max: ", full_max)
#     plt.plot(scaled_data[:,column])
#     plt.show()
scaled_data = full_np

In [ ]:

model = LSTMLastOutput(9,50,1,0).to(device)
# model = LSTMMultiInputSingleOutput(4,10,1,0).to(device)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

In [ ]:

# train-val split for time series
train_size = int(len(scaled_data[:,0]) * 0.8)
val_size = len(scaled_data[:,0])  - train_size
train, val = scaled_data[:train_size,:], scaled_data[train_size:]
if isinstance(model,LSTMLastOutput):
    X_train, y_train = create_dataset_one_output(train, lookback=lookback)
    X_val, y_val = create_dataset_one_output(val, lookback=lookback)
train_loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=False, batch_size=256)
val_loader = data.DataLoader(data.TensorDataset(X_val, y_val), shuffle=False, batch_size=256)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)


In [123]:
optimizer = optim.Adam(model.parameters(),lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',0.5,40,0.000001, verbose=True)
loss_fn = nn.MSELoss()
n_epochs = 20000
best_loss = 100
output_length = scaled_data.shape[0]
y_full_preds = np.ones((output_length-lookback,1))*np.nan
for epoch in tqdm(range(n_epochs)):
    model.train()
    train_plot = np.ones_like(scaled_data[:,-1]) * np.nan
    y_pred_train = []
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_pred = model(X_batch)
        loss = torch.sqrt(loss_fn(y_pred, y_batch))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_rmse = loss
        y_pred_train.extend(y_pred.cpu().detach()[:, -1, :].squeeze().tolist())
    train_plot[lookback:train_size] = y_pred_train
    scheduler.step(train_rmse)
    # Validation
    if epoch % 5 == 0:
        model.eval()
        with torch.no_grad():
            val_plot = np.ones_like(scaled_data[:,-1]) * np.nan
            y_pred_val = []
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = model(X_batch)
                loss = torch.sqrt(loss_fn(y_pred, y_batch))
                val_rmse = loss
                y_pred_val.extend(y_pred.cpu().detach()[:, -1, :].squeeze().tolist())
            val_plot[train_size+lookback:] = y_pred_val
            
            # plot
            fig, ax = plt.subplots()
            ax.plot(scaled_data[:,-1], c='b')
            ax.plot(train_plot, c='r')
            ax.plot(val_plot, c='g')
            start, end = ax.get_xlim()
            # ax.xaxis.set_ticks(np.arange(0, end, 30*24))
            ax.grid()
            if train_rmse<best_loss:
                best_loss = train_rmse
                try:
                    torch.save(model.state_dict(), os.path.join(PATH,"best_model.pt"))
                    plt.savefig(os.path.join(PATH,"result.png"))
                except:
                    os.makedirs(PATH)
                    torch.save(model.state_dict(), os.path.join(PATH,"best_model.pt"))
                    plt.savefig(os.path.join(PATH,"result.png"))

                print("New Best model saved")
            # plt.show()
            plt.close()
        

        print("Epoch %d: train RMSE %.6f, val RMSE %.6f, best val %.6f, Lr: %.7f" % (epoch, train_rmse,  val_rmse, best_loss, optimizer.param_groups[0]["lr"]))

 27%|██▋       | 5316/20000 [1:18:47<3:49:46,  1.07it/s]

New Best model saved
Epoch 5315: train RMSE 0.000175, val RMSE 0.173506, best val 0.000175, Lr: 0.0000031


 27%|██▋       | 5321/20000 [1:18:51<3:31:37,  1.16it/s]

Epoch 5320: train RMSE 0.000418, val RMSE 0.173804, best val 0.000175, Lr: 0.0000031


 27%|██▋       | 5326/20000 [1:18:55<3:43:53,  1.09it/s]

...

Epoch 5390: train RMSE 0.000420, val RMSE 0.174394, best val 0.000172, Lr: 0.0000031


 27%|██▋       | 5396/20000 [1:19:54<3:39:06,  1.11it/s]

New Best model saved
Epoch 5395: train RMSE 0.000172, val RMSE 0.174178, best val 0.000172, Lr: 0.0000031


 27%|██▋       | 5397/20000 [1:19:55<3:36:14,  1.13it/s]


KeyboardInterrupt: 

In [ ]:
model.eval()
with torch.no_grad():
    train_plot = np.ones_like(scaled_data[:,-1]) * np.nan
    y_pred_train = []
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_pred = model(X_batch)
        loss = torch.sqrt(loss_fn(y_pred, y_batch))
        train_rmse = loss
        y_pred_train.extend(y_pred.cpu().detach()[:, -1, :].squeeze().tolist())
    train_plot[lookback:train_size] = y_pred_train
    val_plot = np.ones_like(scaled_data[:,-1]) * np.nan
    y_pred_val = []
    for X_batch, y_batch in val_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_pred = model(X_batch)
        loss = torch.sqrt(loss_fn(y_pred, y_batch))
        val_rmse = loss
        y_pred_val.extend(y_pred.cpu().detach()[:, -1, :].squeeze().tolist())
    val_plot[train_size+lookback:] = y_pred_val

last_data=X_train[-2:,:,:]
with torch.no_grad():
    pred = model(last_data)
    print(pred)
    
last_sample = last_data
last_sample[-2,:,:] = last_data[-1,:,:]
last_sample[-1,:-1,:]=last_data[-1,-1:,:]
last_sample[-1,-1,:] = torch.tensor(historico_btc.tail(1).values.astype('float32')[:,:-1])
test_loader = data.DataLoader(data.TensorDataset(last_sample), shuffle=False, batch_size=1)
test_plot = np.ones([scaled_data[:,-1].shape[0]+1,1]) * np.nan
y_pred_test = []
for X_batch in test_loader:
    X_batch=X_batch[0]
    X_batch = X_batch.to(device)
    y_pred = model(X_batch)
    y_pred_test.extend(y_pred.cpu().detach().tolist())
test_plot[-2:] = [y_pred_test[0][0],y_pred_test[1][0]]
print(y_pred_train[-2:])
print(y_pred_test)
fig, ax = plt.subplots()
ax.plot(scaled_data[:,-1], c='b')
ax.plot(train_plot, c='r')
ax.plot(val_plot, c='g')
ax.plot(test_plot, c='m')
# ax.plot(full_test_plot,c='m')
start, end = ax.get_xlim()
# ax.xaxis.set_ticks(np.arange(0, end, 3))
# ax.yaxis.set_ticks(np.arange(0, 1, 0.05))
ax.grid()
plt.xlim(2170,2200)
# plt.ylim(20000,40000)
plt.show()